# Notebook to create the HAQ input data for only US for the age specific version. This is being done separately cause US is the to be used for the final predictions but US itself is not a "most detailed location"

In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import copy
import matplotlib.pyplot as plt
from db_queries import get_location_metadata
%matplotlib inline

In [2]:
haq_data_dir = '/share/scratch/projects/hssa/haq/HAQ_2017/vanilla_cc_89_paf_242/draws/'
my_data_dir = '/share/scratch/projects/hssa/haq/HAQ_2017/haq_US/'
prog_dir = '/homes/arjuns13/haq/haq/risk_standardized_amen_mort/'
my_data_dir_cancer = '/share/scratch/projects/hssa/haq/HAQ_2017/haq_2017_latest_cancer/cancers_all_most_det_locs/'
years = [2016]#[1990, 1995, 2000, 2005, 2010, 2015, 2016, 2017]

### Location -filtering 195 coutries. HAQ min-max calculations are indeed supposed to be done using all of the 195 countries

In [3]:
locsdf = get_location_metadata(gbd_round_id=5, location_set_id=35)
causelist = pd.read_csv(prog_dir+'amenable_cause_list_GBD.csv')

In [4]:
loc_US = locsdf[locsdf['location_id']==102]['location_id']

In [5]:
loc_US

181    102
Name: location_id, dtype: int64

In [6]:
consolidated_limits  = {
    297:{'age_start':4,
        'age_end':19},
    302:{'age_start':2,
        'age_end':7},
    322:{'age_start':2,
        'age_end':19},
    328:{'age_start':2,
        'age_end':19},
    338:{'age_start':4,
        'age_end':16},
    339:{'age_start':4,
        'age_end':7},
    340:{'age_start':2,
        'age_end':19},
    341:{'age_start':5,
        'age_end':7},
    366:{'age_start':7,
        'age_end':15},
    380:{'age_start':2,
        'age_end':5},
    849:{'age_start':8,
        'age_end':19},
    429:{'age_start':8,
        'age_end':19},
    432:{'age_start':8,
        'age_end':19},
    435:{'age_start':8,
        'age_end':13},
    441:{'age_start':8,
        'age_end':19},
    468:{'age_start':8,
        'age_end':19},
    484:{'age_start':4,
        'age_end':19},
    487:{'age_start':4,
        'age_end':13},
    492:{'age_start':5,
        'age_end':19},
    493:{'age_start':4,
        'age_end':19},
    494:{'age_start':2,
        'age_end':19},
    498:{'age_start':4,
        'age_end':19},
    508:{'age_start':5,
        'age_end':7},
    527:{'age_start':5,
        'age_end':19},
    529:{'age_start':5,
        'age_end':19},
    531:{'age_start':5,
        'age_end':19},
    534:{'age_start':5,
        'age_end':19},
    545:{'age_start':4,
        'age_end':19},
    587:{'age_start':2,
        'age_end':14},
    589:{'age_start':4,
        'age_end':19},
    643:{'age_start':2,
        'age_end':18},
    708:{'age_start':2,
        'age_end':19}
}
cause_limits = pd.DataFrame(consolidated_limits) 
cause_limits = cause_limits.T.reset_index()
cause_limits = cause_limits.rename(index=str, columns={'index':'cause_id'})

In [7]:
len(consolidated_limits)

32

### Combined data - for all most detailed locations

In [8]:
US_data_noncancer =[]

for y in years:
    print('Processing: ',y)
    for i in list(loc_US):
        print('Processing loc: ',i)
        df = pd.read_csv(my_data_dir+'std_all_most_det_locs/'+str(y)+'/'+str(i)+'.csv')
        # dropping cancer
        df = df[~df['cause_id'].isin([849, 429, 432, 435, 441, 468, 484, 487])]
        
        df = df.merge(cause_limits,on='cause_id')
        df = df[(df['age_group_id'] >= df['age_start'])&(df['age_group_id'] <= df['age_end'])]
        df = df.drop(['age_end','age_start'],axis=1)
        
        US_data_noncancer.append(df)
US_data_noncancer = pd.concat(US_data_noncancer, axis=0)

print("Saving to ", my_data_dir+'combined_data/global_data/')
US_data_noncancer.to_csv(my_data_dir+'combined_data/global_data/' + 'US_data_noncancer.csv',index=False)



Processing:  2016
Processing loc:  102
Saving to  /share/scratch/projects/hssa/haq/HAQ_2017/haq_US/combined_data/global_data/


In [9]:
US_data_noncancer.head()

,location_id,year_id,age_group_id,sex_id,cause_id,draw,rsval,val
0,102,2016,2,3,302,0,0.000144,0.000144
1,102,2016,2,3,302,1,0.000140,0.000140
2,102,2016,2,3,302,2,0.000142,0.000142
3,102,2016,2,3,302,3,0.000153,0.000153
4,102,2016,2,3,302,4,0.000147,0.000147


In [10]:
my_data_dir_cancer = '/share/scratch/projects/hssa/haq/HAQ_2017/haq_2017_latest_cancer/cancers_all_most_det_locs/'
US_data_cancer =[]
for y in years:
    print('Processing: ',y)
    for i in list(loc_US):
        print('Processing loc: ',i)
        df = pd.read_csv(my_data_dir_cancer+str(y)+'/'+str(i)+'.csv')
        # keeping only cancer
        df = df[df['cause_id'].isin([849, 429, 432, 435, 441, 468, 484, 487])]
        
        df = df.merge(cause_limits,on='cause_id')
        df = df[(df['age_group_id'] >= df['age_start'])&(df['age_group_id'] <= df['age_end'])]
        df = df.drop(['age_end','age_start'],axis=1)
        
        US_data_cancer.append(df)
US_data_cancer = pd.concat(US_data_cancer, axis=0)

print("Saving to ", my_data_dir+'combined_data/global_data/')
US_data_cancer.to_csv(my_data_dir+'combined_data/global_data/' + 'US_data_cancer.csv',index=False)

Processing:  2016
Processing loc:  102
Saving to  /share/scratch/projects/hssa/haq/HAQ_2017/haq_US/combined_data/global_data/


In [11]:
US_data_cancer.head()

,location_id,year_id,age_group_id,sex_id,cause_id,draw,rsval,val
0,102,2016,8,3,429,0,0.067002,NaN
1,102,2016,9,3,429,0,0.068269,NaN
2,102,2016,10,3,429,0,0.061407,NaN
3,102,2016,11,3,429,0,0.086582,NaN
4,102,2016,12,3,429,0,0.107051,NaN
